In [1]:
import cv2
import numpy as np
import pandas as pd
import glob
import matplotlib.pyplot as plt
import random
from sklearn.model_selection import train_test_split
%matplotlib inline

In [2]:
def generator(data, im_height = 224, im_width = 224):
    while 1:
        for i in xrange(0, len(data)):
            labels = []
            images = np.zeros((5,224,224,3))
            for idx, image_path in enumerate(train_data[5*i:5*(i+1)]):
                img = cv2.imread(image_path)
                img = cv2.resize(img, (im_width, im_height))
                images[idx,:,:,:] = img
                if 'cat' in image_path:
                    labels.append(0)
                else:
                    labels.append(1)

            labels = np.array(labels)
            images = np.swapaxes(images, 1, 3)
            images = np.swapaxes(images, 2, 3)
            yield images, labels

In [3]:
train_data = [image for image in glob.glob('../Data/train/*jpg')]
test_data = [image for image in glob.glob('../Data/train/*jpg')]

In [4]:
train_data, cv_data = train_test_split(train_data, test_size = 0.2, random_state = 5)
train_data, hold_data = train_test_split(train_data, test_size = 0.25, random_state = 5)

In [5]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD

model = Sequential()
model.add(Convolution2D(64, 3, 3, activation='relu',input_shape = (3,224,224), border_mode='same', name='block1_conv1'))
model.add(Convolution2D(64, 3, 3, activation='relu', border_mode='same', name='block1_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block1_pool'))

# Block 2
model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='block2_conv1'))
model.add(Convolution2D(128, 3, 3, activation='relu', border_mode='same', name='block2_conv2'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block2_pool'))

# Block 3
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv1'))
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv2'))
model.add(Convolution2D(256, 3, 3, activation='relu', border_mode='same', name='block3_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block3_pool'))

# Block 4
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv1'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv2'))
model.add( Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block4_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block4_pool'))

# Block 5
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv1'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv2'))
model.add(Convolution2D(512, 3, 3, activation='relu', border_mode='same', name='block5_conv3'))
model.add(MaxPooling2D((2, 2), strides=(2, 2), name='block5_pool'))


Using Theano backend.
Using gpu device 0: GeForce GTX 980M (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5103)


In [6]:
model.load_weights('../Data/vgg16_weights_th_dim_ordering_th_kernels_notop.h5')

In [7]:
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [8]:
model.compile(optimizer=SGD(lr = 0.1, nesterov=True, momentum = 0.9, decay = 1e10-6), loss = 'binary_crossentropy', metrics = ['accuracy'])

In [9]:
model.fit_generator(generator(train_data), samples_per_epoch = 5000,  nb_epoch = 10)

Epoch 1/10
5000/5000 [==============================] - 358s - loss: 8.0008 - acc: 0.4994   
Epoch 2/10
5000/5000 [==============================] - 359s - loss: 7.9673 - acc: 0.5012   
Epoch 3/10
5000/5000 [==============================] - 361s - loss: 7.9331 - acc: 0.5038   
Epoch 4/10


Exception: Input arrays should have the same number of samples as target arrays. Found 5 input samples and 0 target samples.